# ETL

Será feito por ano, gerando cada ano como um arquivo separado pra memória poder aguentar

In [1]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
from tqdm.notebook import tqdm

In [2]:
forecast_path = 'forecasts/2.5'
gridpp_path = 'forecasts/1'
observation_path = 'observation'

In [3]:
def is_csv(x):
    if x[-4:] == '.csv':
        return True
    return False

def read_all_files(path, files):
    df = pd.DataFrame()
    for arquivo in tqdm(files):
        df_aux = pd.read_csv(f'{path}/{arquivo}')
        df = df.append(df_aux)

    return df

def filter_observation_list_by_year(year):
    observation_list = os.listdir(observation_path)
    return list(filter(lambda x: re.findall(f'\_{year}\_', x), observation_list))

def kelvin_to_celsius(k):
    return k - 273

def print_status(df):
    print(df.shape)
    print('='*10)
    print(df.info())

def str_to_datime(x):
    return datetime.strptime(x[:-5].replace('T', ' '), '%Y-%m-%d %H:%M:%S')

In [4]:
desired_columns = ['station_id', 'lat', 'long', 'forecast', 'gridpp', 'observations', 'year', 'month', 'day', 'hour']

In [5]:
filter_observation_list_by_year(2021)

['observation_2021_1.csv',
 'observation_2021_0.csv',
 'observation_2021_2.csv',
 'observation_2021_3.csv']

## 2019

In [6]:
year = 2021

In [7]:
to_transform = ['year', 'month', 'day', 'hour'] # to int

In [8]:
forecast_files = os.listdir(forecast_path)
forecast_files = list(filter(is_csv, forecast_files))

forecasts = read_all_files(forecast_path, forecast_files)
forecasts = forecasts[forecasts.year == year]

forecasts.reset_index(drop=True, inplace=True)
forecasts['forecast'] = forecasts['forecast'].apply(lambda x: kelvin_to_celsius(x))

for item in to_transform:
    forecasts = forecasts.astype({item: 'int32'})

print('transforming datetime')
forecasts['datetime'] = forecasts.apply(lambda x: datetime(x['year'], x['month'], x['day'], x['hour']), axis=1)
print('Done!')
print_status(forecasts)

  0%|          | 0/22 [00:00<?, ?it/s]

transforming datetime
Done!
(1078201, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078201 entries, 0 to 1078200
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   nearest_forecast_long  1078201 non-null  float64       
 1   nearest_forecast_lat   1078201 non-null  float64       
 2   long                   1078201 non-null  float64       
 3   station_id             1078201 non-null  object        
 4   lat                    1078201 non-null  float64       
 5   forecast               1078201 non-null  float64       
 6   indexes                1078201 non-null  object        
 7   year                   1078201 non-null  int32         
 8   month                  1078201 non-null  int32         
 9   day                    1078201 non-null  int32         
 10  hour                   1078201 non-null  int32         
 11  datetime               1078201 non-null  dateti

In [9]:
gridpp_files = os.listdir(gridpp_path)
gridpp_files = list(filter(is_csv, gridpp_files))

gridpp = read_all_files(gridpp_path, gridpp_files)
gridpp = gridpp[gridpp.year == year]

gridpp.reset_index(drop=True, inplace=True)
gridpp['forecast'] = gridpp['forecast'].apply(lambda x: kelvin_to_celsius(x))

for item in to_transform:
    gridpp = gridpp.astype({item: 'int32'})
print('transforming datetime')
gridpp['datetime'] = gridpp.apply(lambda x: datetime(x['year'], x['month'], x['day'], x['hour']), axis=1)
print('Done!')
# Only for gridpp
gridpp.rename(columns={'forecast': 'gridpp'}, inplace=True)

print_status(gridpp)

  0%|          | 0/22 [00:00<?, ?it/s]

transforming datetime
Done!
(1071550, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071550 entries, 0 to 1071549
Data columns (total 14 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   long                   1071550 non-null  float64       
 1   lat                    1071550 non-null  float64       
 2   gridpp                 1071550 non-null  float64       
 3   station_id             1071550 non-null  object        
 4   nearest_forecast_lat   0 non-null        float64       
 5   indexes                1071550 non-null  object        
 6   nearest_forecast_long  0 non-null        float64       
 7   nearest_gridpp_lat     1071550 non-null  float64       
 8   nearest_gridpp_long    1071550 non-null  float64       
 9   year                   1071550 non-null  int32         
 10  month                  1071550 non-null  int32         
 11  day                    1071550 non-null  int32 

In [10]:
observation_files = filter_observation_list_by_year(year)
observation = read_all_files(observation_path, observation_files)

observation['observations'] = observation['observations'].apply(lambda x: x.split(':'))
re_to_extract_numbers = r'\-*\d+\.*\d*'
observation['observations'] = observation['observations'].apply(lambda x: float(re.findall(re_to_extract_numbers, x[-1])[0]))

print('transforming datetime')
observation['datetime'] = observation.referenceTime.apply(lambda x: str_to_datime(x))
print('Done!')
print('transforming Station Id')
observation['sourceId'] = observation['sourceId'].apply(lambda x: x.split(':')[0])
print('Done!')

print_status(observation)


  0%|          | 0/4 [00:00<?, ?it/s]

transforming datetime
Done!
transforming Station Id
Done!
(19737920, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19737920 entries, 0 to 5164411
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   sourceId       object        
 1   referenceTime  object        
 2   observations   float64       
 3   datetime       datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 752.9+ MB
None


### Merge

In [11]:
final_data = forecasts.merge(gridpp[['station_id', 'datetime', 'gridpp']], how='inner', on=['station_id', 'datetime'])
final_data.shape

(1068594, 13)

In [12]:
final_data = final_data.merge(observation, how='inner', left_on=['station_id', 'datetime'], right_on=['sourceId', 'datetime'])
final_data.shape

(923359, 16)

In [13]:
final_data = final_data[desired_columns]
final_data

,station_id,lat,long,forecast,gridpp,observations,year,month,day,hour
0,SN18700,59.9423,10.7200,13.008484,6.950012,6.8,2020,10,16,0
1,SN80740,66.9035,13.6460,2.987488,6.850006,6.7,2020,10,16,0
2,SN61630,62.2583,8.2000,1.916687,0.549988,0.4,2020,10,16,0
3,SN78910,64.6933,12.3295,2.030457,6.729584,7.7,2020,10,16,0
4,SN8140,61.4255,11.0803,7.772156,5.049988,4.9,2020,10,16,0
...,...,...,...,...,...,...,...,...,...,...
923354,SN58900,61.9157,6.5585,6.479767,4.350006,4.2,2020,5,13,12
923355,SN84380,68.3710,17.2438,-5.914764,5.739044,3.7,2020,5,13,12
923356,SN10380,62.5773,11.3518,4.539337,0.850006,0.7,2020,5,13,12
923357,SN97120,69.7563,26.1457,1.860626,2.682861,4.7,2020,5,13,12


In [14]:
final_data.to_csv(f'../data/final_data_{year}.csv', index=False)

In [15]:
observation[observation.sourceId == 'SN61630']

,sourceId,referenceTime,observations,datetime
53598,SN61630,2020-01-01T00:00:00.000Z,0.2,2020-01-01 00:00:00
53599,SN61630,2020-01-01T01:00:00.000Z,0.8,2020-01-01 01:00:00
53600,SN61630,2020-01-01T02:00:00.000Z,0.5,2020-01-01 02:00:00
53601,SN61630,2020-01-01T03:00:00.000Z,2.7,2020-01-01 03:00:00
53602,SN61630,2020-01-01T04:00:00.000Z,2.9,2020-01-01 04:00:00
...,...,...,...,...
62377,SN61630,2020-12-31T19:00:00.000Z,-2.2,2020-12-31 19:00:00
62378,SN61630,2020-12-31T20:00:00.000Z,-2.2,2020-12-31 20:00:00
62379,SN61630,2020-12-31T21:00:00.000Z,-2.2,2020-12-31 21:00:00
62380,SN61630,2020-12-31T22:00:00.000Z,-2.5,2020-12-31 22:00:00


In [16]:
observation.shape

(19737920, 4)